<a href="https://colab.research.google.com/github/lblogan14/master_tensorflow_keras/blob/master/ch9_cnn_tensorflow_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Code Prerequisite

In [31]:
import os

import numpy as np
np.random.seed(123)
print("NumPy:{}".format(np.__version__))

import tensorflow as tf
tf.set_random_seed(123)
print("TensorFlow:{}".format(tf.__version__))

NumPy:1.14.6
TensorFlow:1.12.0


In [32]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My' 'Drive/Colab' 'Notebooks/Mastering_TensorFlow/data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/Mastering_TensorFlow/data


In [33]:
!pip3 install imageio
DATASETSLIB_HOME = '../datasetslib'
import sys
if not DATASETSLIB_HOME in sys.path:
    sys.path.append(DATASETSLIB_HOME)
%reload_ext autoreload
%autoreload 2
import datasetslib

datasetslib.datasets_root = os.path.join(os.path.expanduser('~'),'datasets')

#Understanding convolution
For example, let us assume the kernel matrix is a 2 x 2 matrix, and the input image is a 3 x 3
matrix.

![alt text](https://github.com/lblogan14/master_tensorflow_keras/blob/master/images/conv.JPG?raw=true)

Then the feature map looks like the following:
![alt text](https://github.com/lblogan14/master_tensorflow_keras/blob/master/images/fea_map.JPG?raw=true)

The size of the feature maps is reduced by (kernel_size-1). The size of feature map is:
$$size_{feature_map}=size_{features}-size_{kernel}+1$$

**3D Tensor** - Convolution on 3D tensor is applied on each layer (channel) of the 3D tensor

**stride** - how much the kernel slides across the tensor.
$$size_{feature_map}=\frac{size_{features}-size_{kernel}}{n_{strides}}+1$$

**padding** - used on all sides of the input such that the size of features is increased by double of the padding size to avoid reducing the size of the feature map
$$size_{feature_map}=\frac{size_{features}+2*size_{padding}-size_{kernel}}{n_strides}+1$$

##Convolution Operation in TensorFlow
For example, the `tf.nn.conv2d()` operation,

    tf.nn.conv2d(input,
                 filter,
                 strides,
                 padding,
                 use_cudnn_on_gpu=None,
                 data_format=None,
                 name=None
                 )
                               
`input` and `filter` represent the data tensor of the shape `[batch_size, input_height,
input_width, input_depth]` and kernel tensor of the shape `[filter_height,
filter_width, input_depth, output_depth]`. The `output_depth` in the kernel tensor
represents the number of kernels that should be applied to the input. The `strides` tensor
represents the number of cells to slide in each dimension. The `padding` is VALID or SAME

#Understanding pooling
**Pooling** refers to calculating the aggregate statistic over the regions of the convolved feature
space.

For example, a feature map of shape 3x3 and a pooling region of shape 2x2. Then the max-pooling is applied and shown in the following with stride of [1, 1],

![alt text](https://github.com/lblogan14/master_tensorflow_keras/blob/master/images/maxpool.JPG?raw=true)

Then, after max-pooling layer, we get

![alt text](https://github.com/lblogan14/master_tensorflow_keras/blob/master/images/pool.JPG?raw=true)

Generally, the pooling operation is applied with non-overlapping regions, thus the stride
tensor and the region tensor are set to the same values.

For TensorFlow, `max_pooling` can be called with the following initialziation,

    max_poll(value,
             ksize,
             strides,
             padding,
             data_format='NHWC',
             name=None
             )
             
`value` represents the input tensor of the shape `[batch_size, input_height, input_width, input_depth]`. The pooling operation is performed on rectangular regions of shape `ksize`. These regions are offset by the shape `strides`

#CNN architecture pattern - LeNet
Build CNN model based on LeNet pattern by creating the layers in the following sequence:

1. The input layer
2. The convoluational layer 1 that produces a set of feature maps, with ReLU activation
3. The pooling layer 1 that produces a set of statistically aggregated feature maps
4. The convolutional layer 2 that produces a set of feature maps with ReLU activation
5. The pooling layer 2 that produces a set of statistically aggregated feature maps
6. The fully connected layer that flattens the feature maps with ReLU activation
7. The output layer that produces the output by applying simple linear activation

##LeNet for MNIST data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('mnist', one_hot=True)

x_train = mnist.train.images
x_test = mnist.test.images
y_train = mnist.train.labels
y_test = mnist.test.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models

#LeNet CNN for MNIST with TensorFlow

In [0]:
tf.reset_default_graph()

In [0]:
# parameters
n_classes = 10
n_width = 28
n_height = 28
n_depth = 1
n_inputs = n_width * n_height * n_depth  # total pixels

learning_rate = 0.001
n_epochs = 10
batch_size = 100
n_batches = int(mnist.train.num_examples / batch_size)

# input images in shape [n_samples, n_pixels]
x = tf.placeholder(dtype=tf.float32, name='x', shape=[None, n_inputs])
# output labels
y = tf.placeholder(dtype=tf.float32, name='y', shape=[None, n_classes])

# reshape input to [n_samples, n_width, n_height, n_depth]
x_ = tf.reshape(x, shape=[-1, n_width, n_height, n_depth])

Define the first convolutional layer with 32 kernels of shape 4 x 4, thus producing
32 feature maps.

*   define the weights and biases first


In [0]:
layer1_w = tf.Variable(tf.random_normal(shape=[4,4,n_depth,32], 
                                        stddev=0.1),
                      name='l1_w')
layer1_b = tf.Variable(tf.random_normal([32]),
                      name='l1_b')



*   define the convolutional layer with `tf.nn.conv2d`. Note that `stride` defines the elements by which the kernel tensor should slide in each dimension. The dimension order is determined by `data_format`, which could be either `NHWC` or `NCHW`. Generally, the first and last element in `stride` is set to `1`. `padding` could be `SAME` or `VALID`. Remember to add `relu` activation after calling `tf.nn.conv2d`

In [0]:
layer1_conv = tf.nn.relu(tf.nn.conv2d(x_,
                                      layer1_w,
                                      strides=[1,1,1,1],
                                      padding='SAME'
                                     ) + 
                         layer1_b
                        )

Define the first pooling layer with the `tf.nn.max_pool()` function. `ksize` represents the pooling operation using 2 x 2 x 1 regions, and `strides` represents to slide the regions by 2 x 2 x 1 pixels. Thus the regions do not overlap with each other.

In [0]:
layer1_pool = tf.nn.max_pool(layer1_conv,
                             ksize=[1,2,2,1],
                             strides=[1,2,2,1],
                             padding='SAME')

The first convolution layer produces 32 feature maps of size 28 x 28 x 1, which are
then pooled into data of shape 32 x 14 x 14 x 1.

Now move on to the second convolutional layer,

In [0]:
layer2_w = tf.Variable(tf.random_normal(shape=[4,4,32,64],
                                        stddev=0.1),
                       name='l2_w')
layer2_b = tf.Variable(tf.random_normal([64]),
                       name='l2_b')

layer2_conv = tf.nn.relu(tf.nn.conv2d(layer1_pool,
                                      layer2_w,
                                      strides=[1,1,1,1],
                                      padding='SAME'
                                     ) + 
                         layer2_b
                        )
layer2_pool = tf.nn.max_pool(layer2_conv,
                             ksize=[1,2,2,1],
                             strides=[1,2,2,1],
                             padding='SAME')

The second convolutional layer that takes this data as input and produces
64 feature maps. The output of the second convolution layer is of shape 64 x 14 x 14 x 1, which then
gets pooled into an output of shape 64 x 7 x 7 x 1.

Reshape this output before feeding into the fully connected layer of 1024 neurons
to produce a flattened output of size 1024:

In [0]:
layer3_w = tf.Variable(tf.random_normal(shape=[64*7*7*1, 1024],
                                        stddev=0.1),
                       name='l3_w')
layer3_b = tf.Variable(tf.random_normal([1024]),
                       name='l3_b')

layer3_fc = tf.nn.relu(tf.matmul(tf.reshape(layer2_pool, 
                                            [-1, 64*7*7*1]),
                                 layer3_w) + 
                       layer3_b)

The output of the fully connected layer is fed into a linear output layer with 10 outputs. The softmax function is not applied here because it will be automatically applied in the loss function,

In [0]:
layer4_w = tf.Variable(tf.random_normal(shape=[1024, n_classes],
                                        stddev=0.1),
                       name='l4_w')
layer4_b = tf.Variable(tf.random_normal([n_classes]),
                       name='l4_b')

layer4_out = tf.matmul(layer3_fc, layer4_w) + layer4_b

model = layer4_out

In [11]:
# loss function
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y)
loss = tf.reduce_mean(entropy)

# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Now everything is all set. We can start to train the CNN model,

In [12]:
# train and evaluate
with tf.Session() as tfs:
  tf.global_variables_initializer().run()
  for epoch in range(n_epochs):
    total_loss = 0.0
    for batch in range(n_batches):
      x_batch, y_batch = mnist.train.next_batch(batch_size)
      feed_dict = {x:x_batch, y:y_batch}
      batch_loss, _ = tfs.run([loss, optimizer], feed_dict=feed_dict)
      total_loss += batch_loss
    average_loss = total_loss / n_batches
    print('Epoch: {0:04d}    loss = {1:0.6f}'.format(epoch, average_loss))
  print('Model Trained.')
  
  predictions_check = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
  accuracy = tf.reduce_mean(tf.cast(predictions_check, tf.float32))
  feed_dict = {x:x_test, y:y_test}
  print('Accuracy: ', accuracy.eval(feed_dict=feed_dict))

Epoch: 0000    loss = 2.197315
Epoch: 0001    loss = 0.084367
Epoch: 0002    loss = 0.058146
Epoch: 0003    loss = 0.046801
Epoch: 0004    loss = 0.035882
Epoch: 0005    loss = 0.031906
Epoch: 0006    loss = 0.023579
Epoch: 0007    loss = 0.023606
Epoch: 0008    loss = 0.019154
Epoch: 0009    loss = 0.015893
Model Trained.
Accuracy:  0.9876


#LeNet CNN for MNIST with Keras

In [13]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Reshape
from keras.optimizers import SGD

Using TensorFlow backend.


In [0]:
tf.reset_default_graph()
keras.backend.clear_session()

Define the number of filters for each layer

In [0]:
n_filters = [32, 64]

In [0]:
learning_rate = 0.01
n_epochs = 10
batch_size = 100

Define the sequential model and add the layer to reshape the input data to shape
`(n_width,n_height,n_depth)`:

In [0]:
model = Sequential()

model.add(Reshape(target_shape=(n_width, n_height, n_depth),
                  input_shape=(n_inputs,)
                 )
         )

Add the first convolutional layer with 4 x 4 kernel filter, `SAME` padding and `relu`
activation:

In [0]:
model.add(Conv2D(filters=n_filters[0],
                 kernel_size=4,
                 padding='SAME',
                 activation='relu'
                )
         )

Add the pooling layer with region size of 2 x 2 and stride of 2 x 2:

In [0]:
model.add(MaxPooling2D(pool_size=(2,2),
                       strides=(2,2)
                      )
         )

The second convolutional layer,

In [0]:
model.add(Conv2D(filters=n_filters[1],
                 kernel_size=4,
                 padding='SAME',
                 activation='relu',
                )
         )
model.add(MaxPooling2D(pool_size=(2,2),
                       strides=(2,2)
                      )
         )

Add a layer to flatten the output of the second pooling layer and a fully
connected layer of 1024 neurons to handle the flattened output:

In [0]:
model.add(Flatten())

model.add(Dense(units=1024, activation='relu'))

Add the final output layer with the `softmax` activation:

In [27]:
model.add(Dense(units=n_classes, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        544       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        32832     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              3212288   
__________

Train and evaluate,

In [28]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=learning_rate),
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=batch_size, epochs=n_epochs)

score=model.evaluate(x_test, y_test)
print('\nTest loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
55000/55000 [==============================] - 8s 140us/step - loss: 0.9424 - acc: 0.7544
Epoch 2/10
55000/55000 [==============================] - 7s 133us/step - loss: 0.2490 - acc: 0.9253
Epoch 3/10
55000/55000 [==============================] - 7s 134us/step - loss: 0.1760 - acc: 0.9475
Epoch 4/10
55000/55000 [==============================] - 7s 134us/step - loss: 0.1368 - acc: 0.9596
Epoch 5/10
55000/55000 [==============================] - 7s 134us/step - loss: 0.1125 - acc: 0.9661
Epoch 6/10
55000/55000 [==============================] - 7s 133us/step - loss: 0.0961 - acc: 0.9711
Epoch 7/10
55000/55000 [==============================] - 7s 134us/step - loss: 0.0842 - acc: 0.9751
Epoch 8/10
55000/55000 [==============================] - 7s 133us/step - loss: 0.0751 - acc: 0.9778
Epoch 9/10
55000/55000 [==============================] - 7s 134us/step - loss: 0.0686 - acc: 0.9801
Epoch 10/10
10000/10000 [==============================] - 1s 97us/step

Test loss: 0.06148

#LeNet for CIFAR10 Data
The CIFAR-10 dataset consists of 60,000 RGB color images of the shape 32x32 pixels. The
images are equally divided into 10 different categories or classes: airplane, automobile, bird,
cat, deer, dog, frog, horse, ship, and truck. CIFAR-10 and CIFAR-100 are subsets of a large
image dataset comprising of 80 million images.
## Get the CIFAR10 Data

In [34]:
from datasetslib.cifar import cifar10
from datasetslib import imutil
dataset = cifar10()
dataset.x_layout=imutil.LAYOUT_NHWC
dataset.load_data()
dataset.scaleX()

Downloading: http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Downloaded : /root/datasets/cifar10/cifar-10-python.tar.gz ( 170498071 bytes)
Extracting  /root/datasets/cifar10/cifar-10-python.tar.gz


The data is loaded such that the images are in the `'NHWC'` format, that makes the data
variable of shape `(number_of_samples, image_height, image_width,
image_channels)`. We refer to image channels as image depth. Each pixel in the images is a
number from 0 to 255. The dataset is scaled using MinMax scaling to normalize the images
by dividing all pixel values with 255.

The loaded and pre-processed data becomes available in the dataset object variables as
dataset.`X_train, dataset.Y_train, dataset.X_test, and dataset.Y_test`.

##ConvNets for CIFAR10 with TensorFlow
Keep the layers, filters, and their sizes the same as in the MNIST examples earlier, with
one new addition of regularization layer. 

Since this data set is complex as compared to the
MNIST, we add additional dropout layers for the purpose of regularization:

    tf.nn.dropout(layer, keep_prob)
    
The placeholder `keep_prob` is set to 1 during prediction and evaluation.
That way we can reuse the same model for training as well as prediction
and evaluation.

In [0]:
tf.reset_default_graph()

In [0]:
# parameters
n_filters = [32, 64]
learning_rate = 0.001

# input images of shape = (n_samples, n_width, n_height, n_depth)
x = tf.placeholder(dtype=tf.float32, name='x',
                   shape=[None, dataset.width, dataset.height, dataset.depth])
# target output
y = tf.placeholder(dtype=tf.float32, name='y',
                   shape=[None, dataset.n_classes])

keep_prob = tf.placeholder(tf.float32)

# conv layer 1
layer1_w = tf.Variable(tf.random_normal(shape=[4,4,dataset.depth,n_filters[0]],
                                        stddev=0.01),
                       name='l1_w'
                      )
layer1_b = tf.Variable(tf.random_normal([n_filters[0]]),
                       name='l1_b'
                      )
layer1_conv = tf.nn.relu(tf.nn.conv2d(x,
                                      layer1_w,
                                      strides=[1,1,1,1],
                                      padding='SAME'
                                     ) +
                         layer1_b
                        )
# pooling layer 1
layer1_pool = tf.nn.max_pool(layer1_conv,
                             ksize=[1,2,2,1],
                             strides=[1,2,2,1],
                             padding='SAME')
layer1_pool = tf.nn.dropout(layer1_pool, keep_prob)

# conv layer 2
layer2_w = tf.Variable(tf.random_normal(shape=[4,4,n_filters[0],n_filters[1]],
                                        stddev=0.01),
                       name='l2_w'
                      )
layer2_b = tf.Variable(tf.random_normal([n_filters[1]]),
                       name='l2_b'
                      )
layer2_conv = tf.nn.relu(tf.nn.conv2d(layer1_pool,
                                      layer2_w,
                                      strides=[1,1,1,1],
                                      padding='SAME'
                                     ) +
                         layer2_b
                        )
# pooling layer 2
layer2_pool = tf.nn.max_pool(layer2_conv,
                             ksize=[1,2,2,1],
                             strides=[1,2,2,1],
                             padding='SAME')
layer2_pool = tf.nn.dropout(layer2_pool, keep_prob)

# fully connected layer
layer3_w = tf.Variable(tf.random_normal(shape=[8*8*64, 1024],
                                        stddev=0.01),
                       name='l3_w'
                      )
layer3_b = tf.Variable(tf.random_normal([1024]),
                       name='l3_b')
layer3_fc = tf.nn.relu(tf.matmul(tf.reshape(layer2_pool, [-1, 8*8*64]),
                                 layer3_w) + 
                       layer3_b
                      )
layer3_fc = tf.nn.dropout(layer3_fc, keep_prob)

# output layer
layer4_w = tf.Variable(tf.random_normal(shape=[1024, dataset.n_classes],
                                        stddev=0.01),
                       name='l4_w'
                      )
layer4_b = tf.Variable(tf.random_normal([dataset.n_classes]),
                       name='l4_b'
                      )
layer4_out = tf.matmul(layer3_fc, layer4_w) + layer4_b

model = layer4_out

In [0]:
# loss function
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y)
loss = tf.reduce_mean(entropy)
# optimizer
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(loss)

In [0]:
n_epochs = 20
dataset.y_onehot = True
dataset.batch_size = 128
dataset.batch_shuffle = True
n_batches = int(dataset.n_train/dataset.batch_size)

In [48]:
with tf.Session() as tfs:
  tf.global_variables_initializer().run()
  for epoch in range(n_epochs):
    total_loss = 0.0
    for batch in range(n_batches):
      x_batch, y_batch = dataset.next_batch()
      feed_dict = {x:x_batch, y:y_batch, keep_prob:1.0}
      batch_loss, _ = tfs.run([loss, optimizer], feed_dict=feed_dict)
      total_loss +=batch_loss
    average_loss = total_loss / n_batches
    print("Epoch: {0:04d}   loss = {1:0.6f}".format(epoch,average_loss))
  print("Model Trained.")

  predictions_check = tf.equal(tf.argmax(model,1),tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(predictions_check, tf.float32))
  feed_dict = {x:dataset.X_test, y:dataset.Y_test, keep_prob: 1.0}
  print("Accuracy:", accuracy.eval(feed_dict=feed_dict))

Epoch: 0000   loss = 2.190397
Epoch: 0001   loss = 1.744629
Epoch: 0002   loss = 1.515042
Epoch: 0003   loss = 1.365483
Epoch: 0004   loss = 1.245399
Epoch: 0005   loss = 1.142796
Epoch: 0006   loss = 1.043091
Epoch: 0007   loss = 0.940676
Epoch: 0008   loss = 0.840083
Epoch: 0009   loss = 0.745871
Epoch: 0010   loss = 0.649716
Epoch: 0011   loss = 0.557851
Epoch: 0012   loss = 0.478131
Epoch: 0013   loss = 0.396767
Epoch: 0014   loss = 0.330925
Epoch: 0015   loss = 0.273449
Epoch: 0016   loss = 0.225828
Epoch: 0017   loss = 0.187990
Epoch: 0018   loss = 0.162484
Epoch: 0019   loss = 0.140358
Model Trained.
Accuracy: 0.6315


To increase the accuracy, modify the CNN architecture

##ConvNets for CIFAR10 with Keras
The dropout layer in Keras is

    model.add(Dropout(0.2))

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D, Dense, Flatten, Reshape, Dropout
from keras.optimizers import RMSprop

In [0]:
tf.reset_default_graph()
keras.backend.clear_session()

In [0]:
n_filters = [32, 64]
learning_rate = 0.001
n_epochs = 20
batch_size = 128

In [52]:
model = Sequential()
# conv layer 1
model.add(Conv2D(filters=n_filters[0],
                 kernel_size=4,
                 padding='SAME',
                 activation='relu',
                 input_shape=(dataset.height, dataset.width, dataset.depth)
                )
         )
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# conv layer 2
model.add(Conv2D(filters=n_filters[1],
                 kernel_size=4,
                 padding='SAME',
                 activation='relu'
                )
         )
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=dataset.n_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        1568      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        32832     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
__________

In [53]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=learning_rate),
              metrics=['accuracy'])
model.fit(dataset.X_train, dataset.Y_train, batch_size=dataset.batch_size, epochs=n_epochs)
score = model.evaluate(dataset.X_test, dataset.Y_test)
print('\nTest loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/20
50000/50000 [==============================] - 9s 182us/step - loss: 1.6263 - acc: 0.4191
Epoch 2/20
50000/50000 [==============================] - 9s 175us/step - loss: 1.1698 - acc: 0.5899
Epoch 3/20
50000/50000 [==============================] - 9s 174us/step - loss: 0.9993 - acc: 0.6522
Epoch 4/20
50000/50000 [==============================] - 9s 174us/step - loss: 0.8766 - acc: 0.6954
Epoch 5/20
50000/50000 [==============================] - 9s 175us/step - loss: 0.7719 - acc: 0.7306
Epoch 6/20
50000/50000 [==============================] - 9s 175us/step - loss: 0.6792 - acc: 0.7656
Epoch 7/20
50000/50000 [==============================] - 9s 174us/step - loss: 0.5991 - acc: 0.7915
Epoch 8/20
50000/50000 [==============================] - 9s 174us/step - loss: 0.5288 - acc: 0.8180
Epoch 9/20
50000/50000 [==============================] - 9s 175us/step - loss: 0.4561 - acc: 0.8412
Epoch 10/20
50000/50000 [==============================] - 9s 176us/step - loss: 0.4002 - a